In [1]:
from FASTSim_Package.envs.FASTSim_RL_Environment import *
from utils import *   # import replay buffer for experienced replay and plotting function
from DQN_Agent import *
import pandas as pd
import numpy as np

In [2]:
env = gym.make('FASTSim-v0')
a = env.action_space
print(env.state_space, a)

Box(2,) Discrete(11)


In [3]:
if __name__ == '__main__':
    
    n_simulations = 100  # specify the number of driving simulations wanted
    env = gym.make('FASTSim-v0')  
    agent = Agent(gamma=0.99, lr=0.005, input_dims=2, n_actions=11, batch_size=64, 
                  epsilon=1.0, eps_dec=0.9999995, eps_min=0.0, mem_size=1000000)
        
    scores = []
    eps_history = []

    for i in range(1, n_simulations+1):
        done = False
        score = 0
        state = env.reset()
        env.reset_list()
        while not done:
            action = agent.choose_action(state)
            state_, reward, done, info = env.step(action)
            score += reward
            agent.remember(state, action, reward, state_, done) # store experience
            state = state_   # new state -- state
            agent.learn()   
        
        eps_history.append(agent.epsilon)
        scores.append(score)
    
        #avg_score = np.mean(scores[max(0, i-100):(i+1)])
        avg_score = np.mean(scores[max(0, i-1):(i+1)])
        print(' ------- Driving Simulation Round: {}, Score: {:.2f}, Average Score: {:.2f} ------- '\
              .format(i, score, avg_score))
        
        if i % 10 == 0 and i > 0:
            agent.save_model()

    filename = 'FASTSim.png'
    
    # Results
    df = pd.DataFrame.from_dict(env.output)[['soc','fcKwInAch','fcKwOutAch','essKwOutAch',
                                             'mcMechKwOutAch','auxInKw','mcElecKwInAch','mpgge',
                                             'transKwInAch','mcMechKw4ForcedFc',
                                             'canPowerAllElectrically','fcForcedState']]
    df['speed'] = env.cyc['cycMps'] * 2.23694  # Convert mps to mph 
    print('MPG is: ', df['mpgge'].iloc[-1])

    x = [i+1 for i in range(n_simulations)]
    plot_learning_curve(x, scores, eps_history, filename)

Driving simulation round 1 completed. Assigning outputs and calculating results...
 ------- Driving Simulation Round: 1, Score: 135832.75, Average Score: 135832.75 ------- 


KeyboardInterrupt: 

In [ ]:
# Results
df = pd.DataFrame.from_dict(env.output)[['soc','fcKwInAch','fcKwOutAch','essKwOutAch',
                                         'mcMechKwOutAch','auxInKw','mcElecKwInAch','mpgge',
                                         'transKwInAch','mcMechKw4ForcedFc',
                                         'canPowerAllElectrically','fcForcedState']]
df['speed'] = env.cyc['cycMps'] * 2.23694  # Convert mps to mph 
print('MPG is: ', df['mpgge'].iloc[-1])

x = [i+1 for i in range(n_simulations)]
plot_learning_curve(x, scores, eps_history, filename)

In [ ]:
########################################################################################
#  plot
ax = plt.subplot(3,1,1)
speed_line = df.speed.plot(color='xkcd:pale red', ax=ax, label='Speed')
ax.set_ylabel('Speed [MPH]', weight='bold', color='xkcd:bluish')
ax.tick_params('y', colors='xkcd:bluish')

ax = plt.subplot(3,1,2)
line1 = df.fcKwOutAch.plot(ax=ax, label='kW')
line2 = df.essKwOutAch.plot(ax=ax, label='kW')
ax.set_ylabel('Power [kW]', weight='bold', color='xkcd:bluish')
ax.tick_params('y', colors='xkcd:bluish')
ax.legend(['fuel cell','Li-ion battery'])

ax11 = plt.subplot(3,1,3)
plt.plot(score)
ax11.set_ylabel('reward', weight='bold', color='xkcd:bluish')
# speed_line = df.fuelKgAch.plot(color='xkcd:pale red', ax=ax11, label='Speed')
# ax11.set_ylabel('fuel burned [kg]', weight='bold', color='xkcd:bluish')
ax11.set_xlabel('Cycle Time [s]', weight='bold')
ax11.tick_params('y', colors='xkcd:bluish')